In [1]:
from models.GCN import GCN
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from trainer import train, test
from time import perf_counter

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [3]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, activation):
        super(GraphSAGE, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(in_feats, n_hidden, 'mean')
        self.conv2 = dgl.nn.SAGEConv(n_hidden, n_classes, 'mean')
        self.activation = activation

    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = self.activation(h)
        h = self.conv2(graph, h)
        return h

In [5]:
model = GraphSAGE(in_feats, 16, n_classes, activation)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [6]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [7]:
train(model, g, features, labels, train_mask, val_mask, test_mask, epochs=epochs, loss_fn=loss_fn, optimizer=optimizer, lr=lr, weight_decay=weight_decay, device=device)
test(model, g, features, labels, test_mask)

Epoch 00000 | Loss 1.0985 | Train: 0.7667, Val: 0.5660, Test: 0.5820
Epoch 00010 | Loss 0.2456 | Train: 0.9833, Val: 0.7640, Test: 0.7500
Epoch 00020 | Loss 0.0224 | Train: 1.0000, Val: 0.7680, Test: 0.7590
Epoch 00030 | Loss 0.0036 | Train: 1.0000, Val: 0.7700, Test: 0.7560
Epoch 00040 | Loss 0.0013 | Train: 1.0000, Val: 0.7680, Test: 0.7530
Training time: 4.0988s


0.752